In [1]:
import re
import random
from collections import defaultdict
import numpy as np

In [2]:
def load_and_preprocess_text(filename):
    with open(filename, 'r', encoding='utf-8') as f:
        data = f.read()
    
    data = data.replace('\xa0', ' ') #заменить спец символы
    
    sentences = re.split(r'[.!?]+', data)
    sentences = [s.strip() for s in sentences if len(s.strip()) > 10]
    
    return sentences

In [3]:
def tokenize_text(text):
    text = str(text).lower().strip()
    words = re.findall('[а-яё]+', text)
    return ['<s>'] + words + ['</s>']

In [4]:
def create_ngrams(tokens, n):
    return [tuple(tokens[i:i+n]) for i in range(len(tokens)-n+1)]

In [5]:
class NGramModel:
    def __init__(self, n):
        self.n = n
        self.ngrams = defaultdict(int)
        self.contexts = defaultdict(lambda: defaultdict(int))
        self.vocab = set()
    
    def train(self, texts_tokenized):
        for tokens in texts_tokenized:
            # Добавляем слова в словарь
            self.vocab.update(tokens)
            
            ngrams = create_ngrams(tokens, self.n)
            
            # Подсчет n-граммы и контексты
            for ngram in ngrams:
                self.ngrams[ngram] += 1
                context = ngram[:-1]
                next_word = ngram[-1]
                self.contexts[context][next_word] += 1
    
    def get_next_word_prob(self, context):
        #Вероятность следующего слова
        if context not in self.contexts:
            return None
        
        total = sum(self.contexts[context].values())
        probs = {word: count/total for word, count in self.contexts[context].items()}
        return probs
    
    def generate_word(self, context):
        #Генерация следующего слова
        probs = self.get_next_word_prob(context)
        if probs is None:
            return None
        
        words, probabilities = zip(*probs.items())
        return np.random.choice(words, p=probabilities)
    
    def generate_text(self, max_length=20, start_context=None):
        if start_context is None:
            context = ('<s>',) * (self.n - 1) if self.n > 2 else ('<s>',) #С начала предложения
        else:
            context = start_context
        
        generated = list(context)
        
        for _ in range(max_length):
            next_word = self.generate_word(context)
            if next_word is None or next_word == '</s>':
                break
            
            generated.append(next_word)
            
            # Обновить контекст
            if self.n > 1:
                context = tuple(generated[-(self.n-1):])
        
        # Убрать маркеры
        result = [word for word in generated if word not in ['<s>', '</s>']]
        return ' '.join(result)

In [8]:
def main():
    filename = 'pushkin.txt'
    sentences = load_and_preprocess_text(filename)
    
    print(f"Загружено предложений: {len(sentences)}")
    
    texts_tokenized = [tokenize_text(s) for s in sentences]
    print(f"Токенизировано текстов: {len(texts_tokenized)}")
    
    #n=2
    print("\n=== Биграммная модель (n=2) ===")
    bigram_model = NGramModel(2)
    bigram_model.train(texts_tokenized)
    
    print(f"Размер словаря: {len(bigram_model.vocab)}")
    print(f"Количество биграмм: {len(bigram_model.ngrams)}")
    
    print("\nВывод биграммной моделью:")
    for i in range(3):
        text = bigram_model.generate_text(max_length=10)
        print(f"{i+1}. {text}")
    
    # n=3
    print("\n=== Триграммная модель (n=3) ===")
    trigram_model = NGramModel(3)
    trigram_model.train(texts_tokenized)
    
    print(f"Размер словаря: {len(trigram_model.vocab)}")
    print(f"Количество триграмм: {len(trigram_model.ngrams)}")
    
    # Генерация текста триграммной моделью
    print("\nВывод триграммной моделью:")
    for i in range(3):
        text = trigram_model.generate_text(max_length=10)
        print(f"{i+1}. {text}")
    
    print("\n=== Пример вероятностей ===")
    
    # Для биграмм
    context_bigram = ('люблю',)
    probs_bigram = bigram_model.get_next_word_prob(context_bigram)
    if probs_bigram:
        print(f"Биграммы после 'люблю':")
        for word, prob in sorted(probs_bigram.items(), key=lambda x: -x[1])[:5]:
            print(f"  {word}: {prob:.4f}")
    
    # Для триграмм
    context_trigram = ('я', 'люблю')
    probs_trigram = trigram_model.get_next_word_prob(context_trigram)
    if probs_trigram:
        print(f"Триграммы после 'я люблю':")
        for word, prob in sorted(probs_trigram.items(), key=lambda x: -x[1])[:5]:
            print(f"  {word}: {prob:.4f}")

if __name__ == "__main__":
    main()

Загружено предложений: 7231
Токенизировано текстов: 7231

=== Биграммная модель (n=2) ===
Размер словаря: 25113
Количество биграмм: 85603

Вывод биграммной моделью:
1. обойди все школьнически толки лежащие в их
2. слог дурен темен напыщен и признаюсь над воющим потоком на
3. это жестко ей с смиренным заступом в ногах мне в

=== Триграммная модель (n=3) ===
Размер словаря: 25113
Количество триграмм: 98630

Вывод триграммной моделью:
1. 
2. 
3. 

=== Пример вероятностей ===
Биграммы после 'люблю':
  я: 0.1746
  твой: 0.0635
  </s>: 0.0635
  тебя: 0.0476
  твои: 0.0317
Триграммы после 'я люблю':
  свет: 0.1111
  твой: 0.1111
  вечерний: 0.1111
  сегодня: 0.1111
  в: 0.1111
